## Inicialización de Bibliotecas

In [1]:
from google.colab import drive
from google.colab import files
import os
import glob
import pickle
import torch
import shutil

# 1) Copiar y editar repositorio (Una red de campo de lúpulo cuantificada dispersa para la memoria continua en línea)

## Conectar el codigo con Drive

In [2]:
# Este paso se creo para guardar y ocupar los archivos de guardado del entrenamiento, ya que, debido al extenso tiempo que se demoran en ejecutar algunas celdas y que colab
# tiene un uso limitado de GPU, es necesario ejecutarlo en más de 1 sesión.

In [3]:
# Rescata los datos guardados previamente.
# Ejecuta esta celda una vez al principio de tu sesión.
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

print(" Autoreload activado. Los archivos .py se recargarán automáticamente.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Autoreload activado. Los archivos .py se recargarán automáticamente.


In [4]:
# Define la ruta de tu proyecto en Google Drive
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Results'
os.makedirs(project_path_drive, exist_ok=True)
print(f"Carpeta de proyecto en Drive asegurada en: {project_path_drive}")

# Define la ruta donde se guardan los resultados en Colab
source_folder_colab = '/content/SQHN/results' # O '/content/SQHN/data' si los .data se guardan allí

# Define la carpeta de destino para esta ejecución específica
destination_folder_drive = os.path.join(project_path_drive, 'OnCont_L1_Results')

# El comando de copia (no lo ejecutes todavía, solo déjalo listo)
# Usamos -r para copiar el directorio completo y -v para ver qué se copia
copy_command = f"cp -rv {source_folder_colab} {destination_folder_drive}"

print("\nEl comando para guardar los resultados está listo.")
print(f"Se ejecutará: {copy_command}")

Carpeta de proyecto en Drive asegurada en: /content/drive/MyDrive/SQHN_Project_Results

El comando para guardar los resultados está listo.
Se ejecutará: cp -rv /content/SQHN/results /content/drive/MyDrive/SQHN_Project_Results/OnCont_L1_Results


## Clonación del Repositorio e instalación de archivos adicionales.

In [5]:
# Verificar la GPU
!nvidia-smi

Wed Jul 23 06:40:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
# Clonar el repositorio completo
print("Clonando el repositorio de SQHN...")
!git clone https://github.com/Thefelixctm/TrabajoFinalML.git
print("Repositorio clonado.")

# Movernos al subdirectorio del proyecto
%cd TrabajoFinalML/SQHN

Clonando el repositorio de SQHN...
Cloning into 'TrabajoFinalML'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 259 (delta 91), reused 14 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (259/259), 7.43 MiB | 17.69 MiB/s, done.
Resolving deltas: 100% (91/91), done.
Repositorio clonado.
/content/TrabajoFinalML/SQHN


In [6]:
# Instalar la versión específica de PyTorch
# Comando corregido para instalar una versión que si funcione de Pytorch.
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

print("Instalación de PyTorch (versión moderna) completada.")

Looking in indexes: https://download.pytorch.org/whl/cu121
Instalación de PyTorch (versión moderna) completada.


In [7]:
# Descargar y preparar Tiny ImageNet
# Crear el directorio 'data' si no existe
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Directorio '{data_dir}' creado.")

# Descargar el archivo zip
print("Descargando Tiny ImageNet (esto puede tardar unos minutos)...")
!wget -q http://cs231n.stanford.edu/tiny-imagenet-200.zip -P {data_dir}
print("Descarga completada.")

# Descomprimir el archivo en el directorio 'data'
print("Descomprimiendo el archivo...")
!unzip -q {data_dir}/tiny-imagenet-200.zip -d {data_dir}
print("Descompresión completada.")

Directorio 'data' creado.
Descargando Tiny ImageNet (esto puede tardar unos minutos)...
Descarga completada.
Descomprimiendo el archivo...
Descompresión completada.


In [8]:
# Corregir el código para que descargue los datasets automáticamente
# Lista de archivos a modificar
files_to_patch = glob.glob('**/*.py', recursive=True)

print("Archivos que se van a revisar para parcheo:")
for f in files_to_patch:
    print(f"- {f}")

# --- PARCHE 1: Habilitar descargas automáticas ---
print("\n[Parche 1] Habilitando 'download=True'...")
for file_path in files_to_patch:
    !sed -i 's/download=False/download=True/g' {file_path}

# --- PARCHE 2: Corregir la ruta 'hardcoded' de Tiny ImageNet ---
# La ruta original del autor
hardcoded_path = 'C:/Users/nalon/Documents/PythonScripts/DataSets/tiny-imagenet-200'
# La ruta relativa correcta que hemos creado en Colab
correct_path = './data/tiny-imagenet-200'

print(f"\n[Parche 2] Reemplazando ruta absoluta '{hardcoded_path}' por ruta relativa '{correct_path}'...")
for file_path in files_to_patch:
    # Usamos un delimitador diferente (#) en sed porque las rutas contienen '/'
    # Esto evita tener que escapar cada barra en la ruta
    !sed -i 's#{hardcoded_path}#{correct_path}#g' {file_path}

print("\nParcheo completado.")

Archivos que se van a revisar para parcheo:
- TrabajoFinalML/Inicio.py
- TrabajoFinalML/SQHN/visualize.py
- TrabajoFinalML/SQHN/visualize_mnist_occlusion.py
- TrabajoFinalML/SQHN/aa_online.py
- TrabajoFinalML/SQHN/noisy_aa_online.py
- TrabajoFinalML/SQHN/aa_online_tree_pre.py
- TrabajoFinalML/SQHN/noise_encode_img.py
- TrabajoFinalML/SQHN/MHN_layer.py
- TrabajoFinalML/SQHN/trainer.py
- TrabajoFinalML/SQHN/plot_images.py
- TrabajoFinalML/SQHN/recognition_tree.py
- TrabajoFinalML/SQHN/infer_compare.py
- TrabajoFinalML/SQHN/aa_online_tree.py
- TrabajoFinalML/SQHN/autoassociate.py
- TrabajoFinalML/SQHN/Tree.py
- TrabajoFinalML/SQHN/aa_onlineBP.py
- TrabajoFinalML/SQHN/MHN_EWC.py
- TrabajoFinalML/SQHN/Unit.py
- TrabajoFinalML/SQHN/data_loader.py
- TrabajoFinalML/SQHN/visualize_custom.py
- TrabajoFinalML/SQHN/qr_final_comparison.py
- TrabajoFinalML/SQHN/plot_image_grid.py
- TrabajoFinalML/SQHN/utilities.py
- TrabajoFinalML/SQHN/qr_restorer_fair_fight.py
- TrabajoFinalML/SQHN/MHN.py
- Trabajo

## Carga de datos guardados en Drive.

In [9]:
# Sincronizar Drive -> Colab
print("Restaurando resultados previos desde Google Drive...")

# Define las rutas
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
results_source_drive = os.path.join(project_path_drive, 'OnCont_L1_Results')
results_dest_colab = '/content/SQHN/results' # Directorio de checkpoints

# Asegurarse de que los directorios existen
!mkdir -p {results_source_drive}
!mkdir -p {results_dest_colab}

# Copiar los archivos .data existentes desde Drive a Colab
# -v: verbose, -n: no-clobber (no sobrescribir si ya existe)
!cp -vn {results_source_drive}/*.data {results_dest_colab}/

print("Sincronización inicial completada.")

Restaurando resultados previos desde Google Drive...
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt0_hdsz2000.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt0_hdsz2000.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt0_hdsz3300.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt0_hdsz3300.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt0_hdsz700.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt0_hdsz700.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt1_hdsz2000.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt1_hdsz2000.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt1_hdsz3300.data' -> '/content/SQHN/results/AA_OnlineBP_data4_opt1_hdsz3300.data'
'/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Results/AA_OnlineBP_data4_opt1_hdsz700.data' -> '/content/SQHN/results/AA_OnlineB

## Entrenamiento y descarga de los Gráficos del paper.

In [11]:
print("Ejecutando el test de aprendizaje online-continual (OnCont-L1)...")
!python main.py --test OnCont-L1 --drive_path {destination_drive}

print("Guardando los resultados de 'OnCont-L1' en Google Drive...")

# Definimos las rutas de nuevo para estar seguros
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
results_source_colab = '/content/SQHN/results'
destination_drive = os.path.join(project_path_drive, 'OnCont_L1_Results')

# Copiar la carpeta de resultados
!cp -rv {results_source_colab} {destination_drive}

print(f"¡Éxito! Resultados guardados en {destination_drive}")

Ejecutando el test de aprendizaje online-continual (OnCont-L1)...
--- Iniciando bloque de tests: online_L1 (Online Learning) ---

[Sub-bloque 1/3] EMNIST
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_Online_Simf2_numN300_data6_numData3000_upType0_det0.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt0_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt1_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt2_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt3_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AE_Online_data6_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_Online_Simf2_numN1300_data6_numData3000_upType0_det0.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt0_hdsz1300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_

In [ ]:
# Generar los Gráficos
# Ahora que los resultados están a salvo, puedes generar los gráficos.
# Si la sesión se desconecta, en la próxima sesión puedes saltar directamente
# a esta celda después de montar Drive y restaurar los resultados.

print("Generando los gráficos para 'OnCont-L1'...")
!python main.py --plot OnCont-L1

# Copiar también los gráficos generados a Drive
plots_source_colab = '/content/SQHN/plots' # Asumiendo que se guardan aquí
destination_plots_drive = os.path.join(project_path_drive, 'OnCont_L1_Plots')
!cp -rv {plots_source_colab} {destination_plots_drive}

print(f"Gráficos guardados en {destination_plots_drive}")

Generando los gráficos para 'OnCont-L1'...
✅ Gráfico guardado localmente en: plots/OnCont_L1_Sensitivity.png
Figure(600x150)
/content/SQHN/plot.py:462: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  axs[0,d].errorbar(x, torch.mean(dta[3], dim=0), yerr=torch.std(dta[3], dim=0), fmt='o', alpha=.6, ls='-', label=names[o], markersize=2.5, color=clrs[o], marker='s')
/content/SQHN/plot.py:463: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  axs[1,d].errorbar(x, torch.mean(dta[2], dim=0), yerr=torch.std(dta[2], dim=0), fmt='o', alpha=.6, ls='-', label=names[o], markersize=2.5, color=clrs[o], marker='s')
/content/SQHN/plot.py:478: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will 

# 2) Implementación de Autoencoder.

## Entrenamiento de Autoencoder e implementación de Gráficos de comparación.

In [12]:
%load_ext autoreload
%autoreload 2

print(" Autoreload activado. Los archivos .py se recargarán automáticamente.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 Autoreload activado. Los archivos .py se recargarán automáticamente.


In [13]:
print("   Iniciando el test de aprendizaje 'OnCont-L1'...")
print("   El script saltará los trabajos completados y ejecutará los que faltan.")
print("   Cada resultado se guardará en Drive de forma inmediata.")

!python main.py --test OnCont-L1 --drive_path {destination_drive_results}

print("\n ¡Todos los entrenamientos del test 'OnCont-L1' han finalizado!")

   Iniciando el test de aprendizaje 'OnCont-L1'...
   El script saltará los trabajos completados y ejecutará los que faltan.
   Cada resultado se guardará en Drive de forma inmediata.
--- Iniciando bloque de tests: online_L1 (Online Learning) ---

[Sub-bloque 1/3] EMNIST
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_Online_Simf2_numN300_data6_numData3000_upType0_det0.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt0_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt1_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt2_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_OnlineBP_data6_opt3_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AE_Online_data6_hdsz300.data'
✔️ SALTANDO: El archivo ya existe en Colab en 'results/AA_Online_Simf2_numN1300_data6_numData3000_upType0_det0.data'
✔️ SALTANDO: El archivo ya existe en Colab

In [15]:
# Generar y Guardar los Gráficos (Versión Robusta)
# 1. Redefinir las rutas clave para asegurarnos de que existen en esta celda
project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
destination_drive_plots = os.path.join(project_path_drive, 'OnCont_L1_Plots')
plots_source_colab = '/content/SQHN/plots'

# 2. Asegurarse de que la carpeta de destino en Drive existe
os.makedirs(destination_drive_plots, exist_ok=True)

# 3. Generar los gráficos (esto ya sabes que funciona)
print(" Generando los gráficos para 'OnCont-L1'...")
!python main.py --plot OnCont-L1

# 4. Copiar los archivos a Drive usando f-string para garantizar la correcta
#    sustitución de las variables.
print("\nCopiando los gráficos generados a Google Drive...")
!cp -rv {plots_source_colab}/* {destination_drive_plots}/

print(f"\n ¡ÉXITO! Gráficos guardados en: {destination_drive_plots}")

 Generando los gráficos para 'OnCont-L1'...
✅ Gráfico guardado localmente en: plots/OnCont_L1_Sensitivity.png
Figure(600x150)
/content/SQHN/plot.py:462: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  axs[0,d].errorbar(x, torch.mean(dta[3], dim=0), yerr=torch.std(dta[3], dim=0), fmt='o', alpha=.6, ls='-', label=names[o], markersize=2.5, color=clrs[o], marker='s')
/content/SQHN/plot.py:463: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  axs[1,d].errorbar(x, torch.mean(dta[2], dim=0), yerr=torch.std(dta[2], dim=0), fmt='o', alpha=.6, ls='-', label=names[o], markersize=2.5, color=clrs[o], marker='s')
/content/SQHN/plot.py:478: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will

## Comprobación del Autoencoder.

In [17]:
# SQHN (el más grande de EMNIST)
filename_sqhn = 'results/AA_OnlineCont_Simf2_numN2300_data6_numData3000_upType0_det0.data'

# Autoencoder (el más grande de EMNIST)
filename_ae = 'results/AE_Online_data6_hdsz2300.data'

def inspect_results(filename):
    """
    Carga un archivo de resultados y muestra los valores finales de MSE y Acc.
    """
    print(f"--- Inspeccionando el archivo: {filename} ---")
    try:
        with open(filename, 'rb') as filehandle:
            data = pickle.load(filehandle)

        # En nuestros archivos .data:
        # data[2] es el tensor de recall_mse_n
        # data[3] es el tensor de recall_pcnt_n

        mse_tensor = data[2]
        acc_tensor = data[3]

        # Calculamos la media del rendimiento final (la última columna de cada tensor)
        final_mse_mean = torch.mean(mse_tensor[:, -1])
        final_mse_std = torch.std(mse_tensor[:, -1])

        final_acc_mean = torch.mean(acc_tensor[:, -1])
        final_acc_std = torch.std(acc_tensor[:, -1])

        print(f"  -> MSE Final Promedio: {final_mse_mean.item():.6f} (std: {final_mse_std.item():.6f})")
        print(f"  -> Acc Final Promedio: {final_acc_mean.item():.6f} (std: {final_acc_std.item():.6f})")
        print("-" * 40 + "\n")

    except FileNotFoundError:
        print(f"  -> ERROR: No se encontró el archivo. Verifica el nombre.")
    except Exception as e:
        print(f"  -> Ocurrió un error: {e}")

# Ejecutamos la inspección
inspect_results(filename_sqhn)
inspect_results(filename_ae)

--- Inspeccionando el archivo: results/AA_OnlineCont_Simf2_numN2300_data6_numData3000_upType0_det0.data ---
  -> MSE Final Promedio: 0.004538 (std: 0.000126)
  -> Acc Final Promedio: 0.809327 (std: 0.007265)
----------------------------------------

--- Inspeccionando el archivo: results/AE_Online_data6_hdsz2300.data ---
  -> MSE Final Promedio: 0.000000 (std: 0.000000)
  -> Acc Final Promedio: 0.000000 (std: 0.000000)
----------------------------------------



## Test de Memoria "One-Shot"

In [19]:
print("  Generando comparación One-Shot (Test de Memoria Perfecta)...")
# Este script muestra cómo el SQHN puede memorizar y reconstruir una
# única imagen a la perfección, mostrando su capacidad de recuerdo.
!python visualize_oneshot.py --num_images 8 --dataset_id 0 --epochs 200

# Copiar la imagen generada a Drive
plots_source_colab = '/content/SQHN/plots'
# ... (código para copiar el archivo ..._oneshot_...) ...
print("\nCopiando visualización One-Shot a Google Drive...")
!cp -rv {plots_source_colab}/reconstruction_oneshot_*.png {destination_drive_plots}/

  Generando comparación One-Shot (Test de Memoria Perfecta)...
Cargando 8 imágenes del dataset ID: 0...
Generando reconstrucciones de SQHN (una memoria por imagen)...
Entrenando Autoencoder por 200 épocas...
Generando visualización One-Shot...
✅ Visualización guardada localmente en: plots/reconstruction_oneshot_comparison_data0_8imgs.png
Figure(600x1600)

Copiando visualización One-Shot a Google Drive...
'/content/SQHN/plots/reconstruction_oneshot_comparison_data0_8imgs.png' -> '/content/drive/MyDrive/SQHN_Project_Final/OnCont_L1_Plots/reconstruction_oneshot_comparison_data0_8imgs.png'


## Comprobación con Codigos QR.

In [20]:
# Cambia este número si quieres probar con otra resolución en el futuro (ej. 128).
RESOLUTION = 96

# Preparar el Entorno
# Crear las carpetas para las imágenes.
clean_folder = "qr_codes_clean"
corrupted_folder = "qr_codes_corrupted"
os.makedirs(clean_folder, exist_ok=True)
os.makedirs(corrupted_folder, exist_ok=True)

print(f" Carpetas listas para la demostración a {RESOLUTION}x{RESOLUTION}px.")
print("-" * 60)

# Subir las Imágenes de Referencia (Limpias)
print(f"PASO 1: Por favor, sube tus imágenes de QR originales y limpias.")
print("(Puedes seleccionar varios archivos a la vez)")

# Mover los archivos subidos a la carpeta correcta.
uploaded_clean = files.upload()
if not uploaded_clean:
    print("\n No se subieron archivos limpios. El proceso se ha detenido.")
else:
    # Limpiar la carpeta de archivos antiguos antes de añadir los nuevos.
    for f in os.listdir(clean_folder): os.remove(os.path.join(clean_folder, f))
    for filename, content in uploaded_clean.items():
        shutil.move(filename, os.path.join(clean_folder, filename))
    print(f"\n {len(uploaded_clean)} imágenes limpias listas en la carpeta '{clean_folder}'.")
    print("-" * 60)

    # Construir la Biblioteca de Modelos
    print("\n PASO 2: Construyendo la biblioteca de modelos...")

    # Borramos la biblioteca antigua para forzar el reentrenamiento
    # con la nueva resolución.
    library_file = "qr_model_library.pkl"
    if os.path.exists(library_file):
        print(f"   -> Borrando la biblioteca de modelos antigua ('{library_file}')...")
        os.remove(library_file)

    !python qr_restorer_hires.py learn {clean_folder} --resolution {RESOLUTION}
    print("-" * 60)

    # Subir la Imagen Corrupta a Restaurar
    print(f"\nPASO 3: Ahora, por favor, sube la imagen de QR corrupta que quieres restaurar.")
    print("(Selecciona solo un archivo)")

    uploaded_corrupted = files.upload()
    if not uploaded_corrupted:
        print("\n No se subió ningún archivo corrupto. El proceso se ha detenido.")
    else:
        corrupted_filename = list(uploaded_corrupted.keys())[0]
        # Limpiar la carpeta de corruptos para evitar confusiones.
        for f in os.listdir(corrupted_folder): os.remove(os.path.join(corrupted_folder, f))
        shutil.move(corrupted_filename, os.path.join(corrupted_folder, corrupted_filename))
        image_to_restore_path = os.path.join(corrupted_folder, corrupted_filename)
        print(f"\n Imagen corrupta '{corrupted_filename}' lista para ser restaurada.")
        print("-" * 60)

        # Restaurar la Imagen
        print(f"\n  PASO 4: Iniciando la restauración a {RESOLUTION}x{RESOLUTION}px...")
        !python qr_restorer_hires.py restore {image_to_restore_path} --resolution {RESOLUTION}

        # Guardar el Resultado en Google Drive
        project_path_drive = '/content/drive/MyDrive/SQHN_Project_Final'
        destination_drive_plots = os.path.join(project_path_drive, 'Restauraciones_QR_HiRes')
        os.makedirs(destination_drive_plots, exist_ok=True)

        plots_source_colab = '/content/SQHN/plots'
        output_filename = f"qr_restoration_{RESOLUTION}px_result.png"
        source_file_path = os.path.join(plots_source_colab, output_filename)

        if os.path.exists(source_file_path):
            print("\nPASO 5: Copiando la imagen final a Google Drive...")
            !cp -v {source_file_path} {destination_drive_plots}/
            print(f"\n ¡ÉXITO TOTAL! Visualización guardada en: {destination_drive_plots}")
        else:
            print("\n No se encontró el archivo de resultado para copiar a Drive.")

 Carpetas listas para la demostración a 96x96px.
------------------------------------------------------------
PASO 1: Por favor, sube tus imágenes de QR originales y limpias.
(Puedes seleccionar varios archivos a la vez)


Saving qr_lote_a.png to qr_lote_a.png
Saving qr_lote_b.png to qr_lote_b.png
Saving qr_lote_c.png to qr_lote_c.png
Saving qr_lote_d.png to qr_lote_d.png
Saving qr_lote_e.png to qr_lote_e.png
Saving qr_lote_f.png to qr_lote_f.png
Saving qr_lote_g.png to qr_lote_g.png
Saving qr_lote_h.png to qr_lote_h.png
Saving qr_lote_j.png to qr_lote_j.png
Saving qr_lote_k.png to qr_lote_k.png

 10 imágenes limpias listas en la carpeta 'qr_codes_clean'.
------------------------------------------------------------

 PASO 2: Construyendo la biblioteca de modelos...
Construyendo biblioteca de modelos (96x96px) desde: qr_codes_clean
  -> Entrenando modelos para 'qr_lote_e.png'...
  -> Entrenando modelos para 'qr_lote_c.png'...
  -> Entrenando modelos para 'qr_lote_h.png'...
  -> Entrenando modelos para 'qr_lote_b.png'...
  -> Entrenando modelos para 'qr_lote_g.png'...
  -> Entrenando modelos para 'qr_lote_k.png'...
  -> Entrenando modelos para 'qr_lote_a.png'...
  -> Entrenando modelos para 'qr_lote_j.png'

Saving qr_lote_f_corrupto.png to qr_lote_f_corrupto.png

 Imagen corrupta 'qr_lote_f_corrupto.png' lista para ser restaurada.
------------------------------------------------------------

  PASO 4: Iniciando la restauración a 96x96px...
Intentando restaurar (96x96px): qr_codes_corrupted/qr_lote_f_corrupto.png
  -> SQHN recordó el patrón de: 'qr_lote_f.png'
  -> Autoencoder (pelea justa) encontró la mejor coincidencia con: 'qr_lote_f.png'

✅ Visualización guardada localmente en: plots/qr_restoration_96px_result.png
Figure(1200x400)

PASO 5: Copiando la imagen final a Google Drive...
'/content/SQHN/plots/qr_restoration_96px_result.png' -> '/content/drive/MyDrive/SQHN_Project_Final/Restauraciones_QR_HiRes/qr_restoration_96px_result.png'

 ¡ÉXITO TOTAL! Visualización guardada en: /content/drive/MyDrive/SQHN_Project_Final/Restauraciones_QR_HiRes
